In [ ]:
import cftime
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

import src.tools as tools

## Debug nc files

In [ ]:
ds = xr.open_dataset('fram_data/roho160_rst.nc')
ds

In [ ]:
salt = ds.temp.isel(ocean_time=0, s_rho=39)
salt

## Grid

In [ ]:
grid_ds = xr.open_dataset('fram_data/norfjords_160m_grid.nc')
grid_ds

In [ ]:
grid_ds.mask_rho.plot()

## Initial conditions

In [ ]:
init_ds = xr.open_dataset('fram_data/roho160_init_GLORYS_20170115_to_20191130.nc')
init_ds.coords["z_rho"] = tools.roho160_tranform_to_z(init_ds)
init_ds

In [ ]:
init_ds['zeta'].isel(ocean_time=0).plot()

In [ ]:
time_cftime = cftime.datetime(2017, 1, 15)
init_ds = init_ds.assign_coords(ocean_time=[time_cftime])

In [ ]:
# init_ds.to_netcdf(path='fram_data/roho160_init_GLORYS_20170115_to_20191130_fix_time_coord.nc', format='NETCDF4')

In [ ]:
init_ds.salt.isel(xi_rho=0, ocean_time=0).plot()

In [ ]:
section = init_ds.salt.isel(xi_rho=0, eta_rho=slice(10, 200), ocean_time=0)
section.plot(x="eta_rho", y="z_rho", figsize=(15, 6), clim=(25, 35))

In [ ]:
init_ds.salt.isel(s_rho=1, ocean_time=0).plot(x="xi_rho", y="eta_rho")

## Boundary conditions

In [ ]:
boundold_ds = xr.open_dataset('fram_data/roho160_bry_GLORYS_20170115_to_20191130.nc')
boundold_ds

In [ ]:
boundary_ds = xr.open_dataset('fram_data/roho160_bry.nc')
boundary_ds

In [ ]:
boundary_ds.zeta_east.isel(ocean_time=0).plot()

In [ ]:
salt_south = boundary_ds.salt_south.isel(ocean_time=0)
salt_south.plot()

## River

In [ ]:
river_ds = xr.open_dataset('fram_data/norfjords_160m_river.nc')

In [ ]:
river_ds

In [ ]:
# Creating new array to fill the gaps
v_data = np.zeros((5, 175))
v_add = xr.DataArray(v_data, dims=river_ds['river_Vshape'].dims, coords=river_ds['river_Vshape'].coords)
temp_data = np.ones((4019, 5, 175)) * 5
t_add = xr.DataArray(temp_data, dims=river_ds['river_temp'].dims, coords=river_ds['river_temp'].coords)
salt_data = np.ones((4019, 5, 175))
s_add = xr.DataArray(salt_data, dims=river_ds['river_salt'].dims, coords=river_ds['river_salt'].coords)

In [ ]:
# New ready data arrays
river_Vshape = xr.concat([v_add, river_ds['river_Vshape']], dim="s_rho")
river_temp = xr.concat([t_add, river_ds['river_temp']], dim="s_rho")
river_salt = xr.concat([s_add, river_ds['river_salt']], dim="s_rho")

In [ ]:
# Remove old and add new variables with new s_rho=40 dimension
river_ds = river_ds.drop_dims('s_rho')
river_ds = river_ds.assign(river_Vshape=river_Vshape, river_temp=river_temp, river_salt=river_salt)

In [ ]:
river_ds

In [ ]:
river_ds.to_netcdf(path='fram_data/norfjords_160m_river_new.nc', format='NETCDF4')

## Tides

In [ ]:
tide = xr.open_dataset('fram_data/tide.nc')
tide

## Output files

In [ ]:
ds = xr.open_dataset('fram_data/roho160_his_0001.nc')
ds

In [ ]:
ds['temp'].isel(ocean_time=1, s_rho=-1).plot()

In [ ]:
ds.zeta.isel(ocean_time=0).plot()

In [ ]:
da = ds.zeta.isel(ocean_time=1)

In [ ]:
da_stacked = da.where(da<-1).stack(x=['eta_rho','xi_rho'])
da_stacked[da_stacked.notnull()]

In [ ]:
da.where(da<-1).isel(eta_rho=slice(690, 701), xi_rho=slice(1430, 1441)).plot()